In [9]:
import pandas as pd
import numpy as np

from opensoundscape.spectrogram import Spectrogram
from opensoundscape.audio import Audio

import IPython.display as ipd


def plot_clip(audio_path,
              st = None,
              end = None,
              bandpass = [1, 10000], 
              mark_at_s = None,
              buffer = None):
    """ Load file, display spectograms and play audio
    
    Args:
        audio_path (str): Audio file path
        directory (str, optional): In case [audio_path] is not a full path, search [directory] for it. Defaults to None.
        bandpass (list, Hz): Length 2 list specifying a frequency range to display. Format is [lower_band, higher_band].
        mark_at_s (list, s): List of seconds to add vertical lines in spectrogram. Typically used to mark start and end of valid clip.
        buffer (float, optional): Add buffer to beginning and end of clip (seconds). Defaults to None.
    """
    
    # Apply buffer if provided
    if buffer and st is not None and end is not None:
        st_buffered = max(0, st - buffer)
        end_buffered = end + buffer
        dur = end_buffered - st_buffered
        audio = Audio.from_file(audio_path, offset=st_buffered, duration=dur).bandpass(bandpass[0], bandpass[1], order=10)
        
        # Automatically mark original clip boundaries when buffer is used
        if mark_at_s is None:
            # Calculate relative positions of original boundaries within buffered audio
            original_st_relative = st - st_buffered
            original_end_relative = end - st_buffered
            mark_at_s = [original_st_relative, original_end_relative]
        else:
            # If mark_at_s is provided, adjust it relative to the buffered start
            mark_at_s = [m - st_buffered for m in mark_at_s]
    else:
        # Original behavior
        dur = end - st
        audio = Audio.from_file(audio_path, offset=st, duration=dur).bandpass(bandpass[0], bandpass[1], order=10)
    
    # Add length markings 
    if mark_at_s is not None:
        for s in mark_at_s:
            plt.axvline(x=s, color='b')
    
    ipd.display(Spectrogram.from_audio(audio).bandpass(bandpass[0], bandpass[1]).plot())
    ipd.display(ipd.Audio(audio.samples, rate=audio.sample_rate, autoplay=True))



In [6]:
df = pd.read_csv('/Users/lviotti/Library/CloudStorage/Dropbox/Work/Kitzes/projects/omao/predictions/perch-predictions-kipu2025-local.csv')
df.head()

,file,file_local,start_time,end_time,omao,apapan,hawama,iiwi,akiapo,hawcre,akepa1,elepai,notes,date,recorder_id
0,/media/kiwi/datasets/unfinalized/kipu2025/2MM0...,/Users/lviotti/Library/CloudStorage/Dropbox/Wo...,315,320,12.571556,-12.264694,-11.746714,-8.279476,-12.594369,-7.123873,-9.500269,-12.528926,NaN,20250126,2MM01186
1,/media/kiwi/datasets/unfinalized/kipu2025/2MM0...,/Users/lviotti/Library/CloudStorage/Dropbox/Wo...,510,515,11.757702,-14.409378,-8.645014,-8.546747,-10.313908,-7.364929,-5.872106,-10.514591,NaN,20250126,2MM01186
2,/media/kiwi/datasets/unfinalized/kipu2025/2MM0...,/Users/lviotti/Library/CloudStorage/Dropbox/Wo...,955,960,11.694942,-9.209687,-8.555916,-5.360497,-7.986407,-2.736204,-1.170177,-10.965650,NaN,20250126,2MM01186
3,/media/kiwi/datasets/unfinalized/kipu2025/2MM0...,/Users/lviotti/Library/CloudStorage/Dropbox/Wo...,365,370,11.690182,-3.828780,-7.413135,-5.028522,-5.548864,-2.098103,-2.047067,-10.968410,NaN,20250126,2MM01186
4,/media/kiwi/datasets/unfinalized/kipu2025/2MM0...,/Users/lviotti/Library/CloudStorage/Dropbox/Wo...,340,345,11.117414,-12.785715,-8.454566,-9.305575,-12.749238,-4.254610,-5.418110,-11.747141,NaN,20250126,2MM01186


In [7]:
row = df.iloc[0]

audio_path = row.file_local
st, end = row.start_time, row.end_time


In [8]:
# Import the bbox widget annotator
from exp.bbox_widget_annotator import create_bbox_widget_annotator

# # Use your existing data
# row = df.iloc[0]  # First row
# audio_path = row.file_local
# st, end = row.start_time, row.end_time

print(f"Creating bbox widget annotator for:")
print(f"Audio: {audio_path}")
print(f"Time: {st:.2f}s - {end:.2f}s")

# Create the annotation interface
annotator, bbox_widget = create_bbox_widget_annotator(
    audio_path=audio_path,
    st=st,
    end=end,
    bandpass=[1, 10000],  # Same as your plot_clip function
    buffer=0.5,  # Add 0.5 second buffer
    classes=['call', 'noise', 'background', 'other']
)

Creating bbox widget annotator for:
Audio: /Users/lviotti/Library/CloudStorage/Dropbox/Work/Kitzes/datasets/kipu2025/2MM01186/Data/2MM01186_20250126_080003.wav
Time: 315.00s - 320.00s


BBoxWidget(classes=['call', 'noise', 'background', 'other'], colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728…


Instructions:
1. Click and drag to draw bounding boxes
2. Select a class from the dropdown for each box
3. Use the widget controls to manage annotations
4. Use get_annotations_df(bbox_widget) to get annotations as DataFrame
